## **Supply Chain Analysis**

In [71]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

plt.style.use('ggplot')

%matplotlib inline

### Task Outline 

- Order Line Analysis

- Service Line Analysis

- Line Lead Time Analysis

- Order Analysis

### Load Datasets

First, let's load all datasets needed for the analysis.

In [ ]:
df_order_lines = pd.read_csv('../data./fact_order_lines.csv', 
                 parse_dates=['order_placement_date', 'agreed_delivery_date', 'actual_delivery_date'])
df_order_lines.head()

In [ ]:
df_order_agg = pd.read_csv('../data./fact_orders_aggregate.csv', 
                 parse_dates=['order_placement_date'])
df_order_agg.head()

In [ ]:
dim_customers = pd.read_csv('../data./dim_customers.csv')
dim_customers.head()

In [ ]:
dim_products = pd.read_csv('../data./dim_products.csv')
dim_products.head()

In [ ]:
dim_target_orders = pd.read_csv('../data./dim_targets_orders.csv')
dim_target_orders.head()

### **Order Lines Analysis**

The following columns will be added by mapping values from different tables to their corresponding key matches in another table. This will create full table which we can now use in our analysis.

In [ ]:
# Merge the customer and product datasets with the order lines dataset
# by performing a left join
# This will allow us to perform further analysis on the dataset

df_order_lines = df_order_lines.merge(dim_customers, how='left', on='customer_id')
df_order_lines = df_order_lines.merge(dim_products, how='left', on='product_id')

df_order_lines.head()

#### **Simple EDA on Dataset**

In [ ]:
df_order_lines.info()

In [ ]:
df_order_lines.duplicated().sum()

There are no null and duplicate values in the dataset. Let's proceed to performing our analysis.

Let's create more columns that we can use for analysis.

In [ ]:
# Also add the week and month of order

df_order_lines['week_no'] = df_order_lines['order_placement_date'].dt.isocalendar().week
df_order_lines['week_no'] = [int(num) for num in df_order_lines['week_no']]
df_order_lines['month'] = df_order_lines['order_placement_date'].dt.month_name()

# We can calculate the number of days it took to deliver an order as well.
df_order_lines['delayed_days']= (df_order_lines['actual_delivery_date'] - df_order_lines['agreed_delivery_date']).dt.days

df_order_lines.head()

### **Metrics Calculation**

In [ ]:
total_order_lines = df_order_lines['order_id'].count()
print(f'Total Order Lines: {total_order_lines}')

In [ ]:
orders_fully_filled = df_order_lines.query("`In Full` == 1").shape[0]
print(f'Total Orders Fully Filled: {orders_fully_filled}')

In [ ]:
line_fill_rate = round(orders_fully_filled/total_order_lines * 100, 2)
print(f'Line fill rate: {line_fill_rate}%')

In [ ]:
total_qty_shipped = df_order_lines['delivery_qty'].sum()
total_qty_ordered = df_order_lines['order_qty'].sum()

volume_fill_rate = round(total_qty_shipped/total_qty_ordered * 100, 2)
print(f'Volume fill rate: {volume_fill_rate}%')

**Customer Insights**

**Total order lines per customer and LIFR % per customer**

In [ ]:
# Get the order lines delivered in full per customer

orders_in_full = df_order_lines.query("`In Full` == 1").groupby('customer_name')[['order_id']].count().reset_index()
orders_in_full = orders_in_full.rename({'order_id':'in_full_orders'}, axis=1)
orders_in_full.head()


In [ ]:
# Get the total order lines delivered per customer
# Let's also add the qty ordered and qty delivered columns as well

orders_per_customer = df_order_lines.groupby('customer_name')[[
    'order_id', 'order_qty', 'delivery_qty']].agg({
        'order_id':'count', 
        'order_qty':'sum', 
        'delivery_qty':'sum'
    }).reset_index()
orders_per_customer = orders_per_customer.rename({'order_id':'total_orders'}, axis=1)
orders_per_customer.head()

In [ ]:
# Merge both datasets

orders_per_customer = orders_per_customer.merge(orders_in_full, how='inner', on='customer_name')
orders_per_customer.head(3)

In [ ]:
# Calculate LIFR % and VOFR %
orders_per_customer['LIFR %'] = round(orders_per_customer['in_full_orders']/orders_per_customer['total_orders'] * 100, 2)
orders_per_customer['VOFR %'] = round(orders_per_customer['delivery_qty']/orders_per_customer['order_qty'] * 100, 2)

orders_per_customer.head(3)

Let's visualize this using a combo chart.

In [ ]:
# Create combo chart
# This will measure the time of the day that is the busiest and its impact on wait time

fig, ax1 = plt.subplots(figsize = (23, 10))

ax1.plot(orders_per_customer['customer_name'], orders_per_customer['total_orders'], color='orange', marker='x')
ax1.set_ylabel('Order Lines', fontsize=15)
ax1.set_ylim(0, 6500)
ax1.grid(False)
ax1.set_xlabel('Customer', fontsize=15)
ax1.legend(['Order Lines'], loc='upper left', fontsize=15)

ax2 = ax1.twinx()
ax2.bar(orders_per_customer['customer_name'], orders_per_customer['LIFR %'], width=0.8, alpha=0.5, color='blue')
ax2.grid(False)
ax2.set_ylabel('LIFR %', fontsize=15)
ax2.set_ylim(0, 85)
ax2.legend(['LIFR %'], loc='upper right', fontsize=15)
plt.title('Total Order Lines vs LIFR % per Customer', fontsize=18)
plt.xticks(orders_per_customer['customer_name'], fontsize=15, rotation=45)


plt.show()

**Product Insights**

In [ ]:
# Get the order lines delivered in full per product

product_in_full = df_order_lines.query("`In Full` == 1").groupby('product_name')[['order_id']].count().reset_index()
product_in_full = product_in_full.rename({'order_id':'in_full_orders'}, axis=1)
product_in_full.head()

In [ ]:
# Get the total order lines delivered per customer
# Let's also add the qty ordered and qty delivered columns as well

orders_per_product = df_order_lines.groupby('product_name')[[
    'order_id', 'order_qty', 'delivery_qty']].agg({
        'order_id':'count', 
        'order_qty':'sum', 
        'delivery_qty':'sum'
    }).reset_index()
orders_per_product = orders_per_product.rename({'order_id':'total_orders'}, axis=1)
orders_per_product.head()

In [ ]:
# Merge both datasets

orders_per_product = orders_per_product.merge(product_in_full, how='inner', on='product_name')
orders_per_product.head(3)

In [ ]:
# Calculate LIFR % and VOFR %
orders_per_product['LIFR %'] = round(orders_per_product['in_full_orders']/orders_per_product['total_orders'] * 100, 2)
orders_per_product['VOFR %'] = round(orders_per_product['delivery_qty']/orders_per_product['order_qty'] * 100, 2)

orders_per_product.head(3)

In [ ]:
# Top 5 products by LIFR% 

top_5_products_by_lifr = orders_per_product.sort_values(by='LIFR %', ascending=False).head().reset_index(drop=True)
top_5_products_by_lifr

In [ ]:
# Plot the top 5 Products by LIFR%

plt.barh(y='product_name', width='LIFR %', data=top_5_products_by_lifr, color='dodgerblue')
plt.title(' Top 5 Products by LIFR %', fontsize=12)
plt.xlabel('LIFR %', fontsize=10)
plt.show()

In [ ]:
# Plot the orders per product category 

category = df_order_lines.groupby('category')[['order_id']].count().reset_index()
plt.bar(x='category', height='order_id', data=category, color='dodgerblue')
plt.title('Order Lines per Product Category')
plt.xlabel('Category')
plt.ylabel('Orders')
plt.show()

**Line Lead Time Analysis**

In [ ]:
# Delayed time by products

delayed_product = df_order_lines.pivot_table(
    values=['order_id'],
    columns=['product_name'],
    index='delayed_days',
    aggfunc='count'
)
delayed_product

In [ ]:
# Delayed time by customer

delayed_customer = df_order_lines.pivot_table(
    values=['order_id'],
    columns=['customer_name'],
    index='delayed_days',
    aggfunc='count'
)
delayed_customer

In [ ]:
# Delayed time by city

delayed_city = df_order_lines.pivot_table(
    values=['order_id'],
    columns=['city'],
    index='delayed_days',
    aggfunc='count'
)
delayed_city


In [ ]:
# Number of orders per delayed day

delayed_days = df_order_lines.groupby(['delayed_days'])[['order_id']].count().reset_index()
delayed_days

In [ ]:
# Make a chart to show how the number of orders per delayed day

plt.barh(y=delayed_days['delayed_days'], width=delayed_days['order_id'], color='dodgerblue')
plt.grid(False)
plt.xlabel('Order Lines')
plt.ylabel('Delayed Days')
plt.title('Number of Order Lines by Delayed Days')

plt.show()

### **Order Analysis**

To perform the order analysis we will use the `df_order_agg` dataset. First we will merge it with `dim_customers` to enable us perform our analysis.

In [ ]:
# Merge dataframes

df_order_agg = df_order_agg.merge(dim_customers, how='inner', on='customer_id')
df_order_agg.head()

In [ ]:
# Let's also add a month and week column to the dataset

df_order_agg['week_no'] = df_order_agg['order_placement_date'].dt.isocalendar().week
df_order_agg['week_no'] = [int(num) for num in df_order_agg['week_no']]
df_order_agg['month'] = df_order_agg['order_placement_date'].dt.month_name()
df_order_agg.head()

### **Metrics Calculation**

In [ ]:
total_orders = df_order_agg['order_id'].count()
print(f'Total Orders: {total_orders}')

In [ ]:
total_on_time_orders = df_order_agg.query("on_time == 1").shape[0]
print(f'Total On Time Orders: {total_on_time_orders}')

In [ ]:
total_in_full_orders = df_order_agg.query("in_full == 1").shape[0]
print(f'Total In Full Orders: {total_in_full_orders}')

In [ ]:
total_otif_orders = df_order_agg.query("otif == 1").shape[0]
print(f'Total Otif Orders: {total_otif_orders}')

**Orders by City and Customer**

In [ ]:
df_order_agg.head()

In [ ]:
# Orders per city

city_orders = df_order_agg.groupby('city')[['order_id']].count().sort_values(by='order_id', ascending=True)
city_orders

In [ ]:
# Orders per customer

customer_orders = df_order_agg.groupby('customer_name')[['order_id']].count().sort_values(by='order_id', ascending=True)
customer_orders = customer_orders.rename({'order_id':'total_orders'}, axis=1).reset_index()
customer_orders

We can also visualize this data.

In [ ]:
# Plot orders per city chart 

city_orders.plot(kind='barh', color='dodgerblue')
plt.title('Orders per City')
plt.xlabel('City')
plt.ylabel('Orders')
plt.legend([])
plt.grid(False)
plt.show()

In [ ]:
# Plot orders per customers 

plt.barh(y='customer_name', width='total_orders', data=customer_orders, color='dodgerblue')
plt.title('Orders per Customer')
plt.xlabel('Customers')
plt.ylabel('Orders')
plt.legend([])
plt.grid(False)
plt.show()

### **Service Level Analysis**

This analysis will take care of the On_time, In_full and On_Time_In_Full analysis of customer and cities. We will like to know which customer is satisfied by our service and who is not so we can make improvements. 

In [ ]:
# Lets find the orders delivered on time per customer

on_time_orders = df_order_agg.query("on_time == 1").groupby('customer_name')[['order_id']].count().reset_index()
on_time_orders = on_time_orders.rename({'order_id':'on_time_orders'}, axis=1)
on_time_orders.head()

In [ ]:
# Lets find the orders delivered in full per customer

in_full_orders = df_order_agg.query("in_full == 1").groupby('customer_name')[['order_id']].count().reset_index()
in_full_orders = in_full_orders.rename({'order_id':'in_full_orders'}, axis=1)
in_full_orders.head()

In [ ]:
# Lets find the orders delivered on time and in full per customer

otif_orders = df_order_agg.query("otif == 1").groupby('customer_name')[['order_id']].count().reset_index()
otif_orders = otif_orders.rename({'order_id':'otif_orders'}, axis=1)
otif_orders.head()

In [ ]:
# Lets merge them to the customer_orders 

customer_orders = customer_orders.merge(on_time_orders, how='left', on='customer_name')
customer_orders = customer_orders.merge(in_full_orders, how='left', on='customer_name')
customer_orders = customer_orders.merge(otif_orders, how='left', on='customer_name')
customer_orders.head()

In [ ]:
# We can calculate the OT%, IF% and OTIF%

customer_orders['OT %'] = round(customer_orders['on_time_orders']/customer_orders['total_orders'] * 100, 2)
customer_orders['IF %'] = round(customer_orders['in_full_orders']/customer_orders['total_orders'] * 100, 2)
customer_orders['OTIF %'] = round(customer_orders['otif_orders']/customer_orders['total_orders'] * 100, 2)

customer_orders.head()

In [ ]:
# Let's calculate the percentage difference of each KPI from the average target

customer_orders['OT_%_diff_from_target'] = round(customer_orders['OT %'] - dim_target_orders['ontime_target%'].mean(), 2)
customer_orders['IF_%_diff_from_target'] = round(customer_orders['IF %'] - dim_target_orders['infull_target%'].mean(), 2)
customer_orders['OTIF_%_diff_from_target'] = round(customer_orders['OTIF %'] - dim_target_orders['otif_target%'].mean(), 2)

customer_orders

**Monthly Trend of KPIs**

In [ ]:
# Firstly let's change the month column to a catergorical data type

months = df_order_agg['month'].unique().tolist()
df_order_agg['month'] = pd.Categorical(df_order_agg['month'], months, ordered=True)
df_order_agg['month'].dtypes

In [ ]:
# Orders per month

month_orders = df_order_agg.groupby('month', observed=False)[['order_id']].count().reset_index()
month_orders = month_orders.rename({'order_id':'total_orders'}, axis=1)
month_orders

In [ ]:
# Plot orders per customers 

plt.plot(month_orders['month'], month_orders['total_orders'], color='dodgerblue')
plt.title('Orders per Month')
plt.xlabel('Month')
plt.ylabel('Orders')
plt.ylim(0, 9000)
plt.legend([])
plt.grid(False)
plt.show()

Let's calculate the KPIs for each month and see the difference from the average target. 

In [ ]:
# Lets find the orders delivered on time per month

month_on_time = df_order_agg.query("on_time == 1").groupby('month', observed=True)[['order_id']].count().reset_index()
month_on_time = month_on_time.rename({'order_id':'on_time_orders'}, axis=1)
month_on_time.head()

In [ ]:
# Lets find the orders delivered in full per customer

month_in_full = df_order_agg.query("in_full == 1").groupby('month', observed=True)[['order_id']].count().reset_index()
month_in_full = month_in_full.rename({'order_id':'in_full_orders'}, axis=1)
month_in_full.head()

In [ ]:
# Lets find the orders delivered on time and in full per customer

month_otif = df_order_agg.query("otif == 1").groupby('month', observed=True)[['order_id']].count().reset_index()
month_otif = month_otif.rename({'order_id':'otif_orders'}, axis=1)
month_otif.head()

In [ ]:
# Merge the dataframes with the month order dataframe

month_orders = month_orders.merge(month_on_time, how='left', on='month')
month_orders = month_orders.merge(month_in_full, how='left', on='month')
month_orders = month_orders.merge(month_otif, how='left', on='month')

month_orders

In [ ]:
# We can calculate the OT%, IF% and OTIF%

month_orders['OT %'] = round(month_orders['on_time_orders']/month_orders['total_orders'] * 100, 2)
month_orders['IF %'] = round(month_orders['in_full_orders']/month_orders['total_orders'] * 100, 2)
month_orders['OTIF %'] = round(month_orders['otif_orders']/month_orders['total_orders'] * 100, 2)

# Let's calculate the percentage difference of each KPI from the average target

month_orders['OT_%_diff_from_target'] = round(month_orders['OT %'] - dim_target_orders['ontime_target%'].mean(), 2)
month_orders['IF_%_diff_from_target'] = round(month_orders['IF %'] - dim_target_orders['infull_target%'].mean(), 2)
month_orders['OTIF_%_diff_from_target'] = round(month_orders['OTIF %'] - dim_target_orders['otif_target%'].mean(), 2)

month_orders

In [ ]:
# Plot OT% per month vs target

plt.plot(month_orders['month'], month_orders['OT %'], color='dodgerblue')
plt.axhline(y=int(dim_target_orders['ontime_target%'].mean()), color='red', linestyle='--')
plt.text(3.5, 87, 'Target', ha='center', va='bottom')
plt.title('OT% per Month')
plt.xlabel('Month')
plt.ylabel('OT %')
plt.ylim(0, 100)
plt.legend([])
plt.grid(False)
plt.show()

In [ ]:
# Plot IF% per month vs target

plt.plot(month_orders['month'], month_orders['IF %'], color='dodgerblue')
plt.axhline(y=int(dim_target_orders['infull_target%'].mean()), color='red', linestyle='--')
plt.text(3.5, 78, 'Target', ha='center', va='bottom')
plt.title('IF% per Month')
plt.xlabel('Month')
plt.ylabel('IF%')
plt.ylim(0, 100)
plt.legend([])
plt.grid(False)
plt.show()

In [ ]:
# Plot IF% per month 

plt.plot(month_orders['month'], month_orders['OTIF %'], color='dodgerblue')
plt.axhline(y=int(dim_target_orders['otif_target%'].mean()), color='red', linestyle='--')
plt.text(3.5, 67, 'Target', ha='center', va='bottom')
plt.title('OTIF% per Month')
plt.xlabel('Month')
plt.ylabel('OTIF%')
plt.ylim(0, 100)
plt.legend([])
plt.grid(False)
plt.show()